In [1]:
import pandas as pd

In [3]:
matches = pd.read_csv('matches.csv', index_col=0)

In [4]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,Match Report,NaN,15.0,7.0,19.1,0.0,0,0,2024,Arsenal
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,Match Report,NaN,13.0,2.0,16.4,0.0,1,1,2024,Arsenal
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,Match Report,NaN,18.0,9.0,13.8,0.0,1,1,2024,Arsenal
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,Match Report,NaN,17.0,5.0,15.0,0.0,0,0,2024,Arsenal
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,Match Report,NaN,13.0,4.0,17.4,0.0,0,0,2024,Arsenal


In [5]:
matches.shape

(1326, 27)

In [7]:
matches['team'].value_counts()

team
Fulham                      67
Newcastle United            67
Nottingham Forest           67
Brentford                   67
Wolverhampton Wanderers     67
Everton                     66
Crystal Palace              66
Bournemouth                 66
Liverpool                   66
Arsenal                     66
Brighton and Hove Albion    66
West Ham United             66
Manchester United           66
Tottenham Hotspur           66
Aston Villa                 66
Manchester City             66
Chelsea                     65
Leicester City              38
Leeds United                38
Southampton                 38
Luton Town                  29
Burnley                     29
Sheffield United            28
Name: count, dtype: int64

In [9]:
matches['round'].value_counts()

round
Matchweek 1     40
Matchweek 15    40
Matchweek 28    40
Matchweek 27    40
Matchweek 25    40
Matchweek 24    40
Matchweek 23    40
Matchweek 22    40
Matchweek 21    40
Matchweek 2     40
Matchweek 19    40
Matchweek 18    40
Matchweek 17    40
Matchweek 16    40
Matchweek 20    40
Matchweek 14    40
Matchweek 7     40
Matchweek 3     40
Matchweek 13    40
Matchweek 5     40
Matchweek 6     40
Matchweek 4     40
Matchweek 8     40
Matchweek 9     40
Matchweek 10    40
Matchweek 11    40
Matchweek 12    40
Matchweek 26    38
Matchweek 29    28
Matchweek 30    20
Matchweek 31    20
Matchweek 33    20
Matchweek 34    20
Matchweek 35    20
Matchweek 36    20
Matchweek 37    20
Matchweek 32    20
Matchweek 38    20
Name: count, dtype: int64

In [10]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes            object
sh              float64
sot             float64
dist            float64
fk              float64
pk                int64
pkatt             int64
season            int64
team             object
dtype: object

In [11]:
matches['date'] = pd.to_datetime(matches['date'])

In [12]:
# atribuir valores numericos a coluna 'venue'
matches['venue_code'] = matches['venue'].astype('category').cat.codes

In [13]:
# atribuir valores numericos a coluna 'opponent'
matches['opp_code'] = matches['opponent'].astype('category').cat.codes

In [14]:
# mudar o horario para apenas os dois primeiros digitos. Ex: 16:30 => 16
matches['hour'] = matches['time'].str.replace(':.+','', regex=True).astype('int')

In [15]:
# atribuir valores numericos ao dia da semana. Ex: monday=1, tuesday=2, ...
matches['day_code'] = matches['date'].dt.dayofweek

In [16]:
matches['target'] = (matches['result'] == 'W').astype('int')

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
# n_estimaotrs, the higher more precise it gets
# min_samples_split the higher less precise
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [23]:
train = matches[matches['date'] < '2024-01-01']

In [24]:
teste = matches[matches['date'] > '2024-01-01']

In [25]:
predictors = ['venue_code', 'opp_code', 'hour', 'day_code']

In [28]:
rf.fit(train[predictors], train['target'])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [30]:
preds = rf.predict((teste[predictors]))

In [31]:
from sklearn.metrics import accuracy_score

In [32]:
acc = accuracy_score(teste['target'], preds)

In [33]:
acc

0.5930232558139535

In [34]:
combined = pd.DataFrame(dict(actual=teste['target'], prediction=preds))

In [35]:
pd.crosstab(index=combined['actual'], columns=combined['prediction'])

prediction,0,1
actual,,
0,80,29
1,41,22


In [36]:
from sklearn.metrics import precision_score

In [37]:
precision_score(teste['target'], preds)

0.43137254901960786

In [38]:
 grouped_matches = matches.groupby('team')

In [40]:
group = grouped_matches.get_group('Manchester City')

In [41]:
group

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3.0,0.0,Burnley,...,0.0,0,0,2024,Manchester City,0,5,20,4,1
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1.0,0.0,Newcastle Utd,...,0.0,0,0,2024,Manchester City,1,16,20,5,1
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Sheffield Utd,...,2.0,0,1,2024,Manchester City,0,18,14,6,1
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5.0,1.0,Fulham,...,0.0,1,1,2024,Manchester City,1,9,15,5,1
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,West Ham,...,1.0,0,0,2024,Manchester City,0,21,15,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,2023-05-06,15:00,Premier League,Matchweek 35,Sat,Home,W,2.0,1.0,Leeds United,...,2.0,0,1,2023,Manchester City,1,10,15,5,1
54,2023-05-14,14:00,Premier League,Matchweek 36,Sun,Away,W,3.0,0.0,Everton,...,2.0,0,0,2023,Manchester City,0,8,14,6,1
56,2023-05-21,16:00,Premier League,Matchweek 37,Sun,Home,W,1.0,0.0,Chelsea,...,0.0,0,0,2023,Manchester City,1,6,16,6,1
57,2023-05-24,20:00,Premier League,Matchweek 32,Wed,Away,D,1.0,1.0,Brighton,...,0.0,0,0,2023,Manchester City,0,4,20,2,0


In [48]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('date')
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [43]:
cols = ['gf', 'ga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt']
new_cols = [f'{c}_rolling' for c in cols]

In [44]:
new_cols

['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [49]:
rolling_averages(group, cols, new_cols)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,...,5,1,3.000000,1.000000,17.666667,6.000000,17.466667,0.666667,0.333333,0.333333
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6.0,0.0,Nott'ham Forest,...,2,1,3.666667,1.666667,19.333333,7.333333,15.933333,0.333333,0.000000,0.000000
6,2022-09-03,17:30,Premier League,Matchweek 6,Sat,Away,D,1.0,1.0,Aston Villa,...,5,0,4.333333,1.666667,18.666667,8.000000,15.033333,0.333333,0.000000,0.000000
9,2022-09-17,12:30,Premier League,Matchweek 8,Sat,Away,W,3.0,0.0,Wolves,...,5,1,3.666667,1.000000,16.000000,6.000000,15.233333,0.333333,0.000000,0.000000
10,2022-10-02,14:00,Premier League,Matchweek 9,Sun,Home,W,6.0,3.0,Manchester Utd,...,6,1,3.333333,0.333333,15.333333,6.666667,17.000000,0.333333,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2024-02-17,17:30,Premier League,Matchweek 25,Sat,Home,D,1.0,1.0,Chelsea,...,5,0,2.666667,0.666667,19.333333,7.333333,17.466667,0.333333,0.000000,0.000000
36,2024-02-20,19:30,Premier League,Matchweek 18,Tue,Home,W,1.0,0.0,Brentford,...,1,1,2.000000,0.666667,25.000000,7.666667,17.500000,0.333333,0.000000,0.000000
37,2024-02-24,17:30,Premier League,Matchweek 26,Sat,Away,W,1.0,0.0,Bournemouth,...,5,1,1.333333,0.333333,25.000000,6.000000,16.966667,0.333333,0.000000,0.000000
39,2024-03-03,15:30,Premier League,Matchweek 27,Sun,Home,W,3.0,1.0,Manchester Utd,...,6,1,1.000000,0.333333,23.666667,7.000000,16.333333,0.666667,0.000000,0.000000


In [50]:
matches_rolling = matches.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))

In [51]:
matches_rolling

date   time            comp         round  \
team                                                                         
Arsenal                 3  2022-08-27  17:30  Premier League   Matchweek 4   
                        4  2022-08-31  19:30  Premier League   Matchweek 5   
                        5  2022-09-04  16:30  Premier League   Matchweek 6   
                        7  2022-09-18  12:00  Premier League   Matchweek 8   
                        8  2022-10-01  12:30  Premier League   Matchweek 9   
...                               ...    ...             ...           ...   
Wolverhampton Wanderers 29 2024-02-17  15:00  Premier League  Matchweek 25   
                        30 2024-02-25  13:30  Premier League  Matchweek 26   
                        32 2024-03-02  15:00  Premier League  Matchweek 27   
                        33 2024-03-09  15:00  Premier League  Matchweek 28   
                        35 2024-03-16  15:00  Premier League  Matchweek 29   

                            day venue result   gf   ga        opponent  ...  \
team                                                                    ...   
Arsenal                 3   Sat  Home      W  2.0  1.0          Fulham  ...   
                        4   Wed  Home      W  2.0  1.0     Aston Villa  ...   
                        5   Sun  Away      L  1.0  3.0  Manchester Utd  ...   
                        7   Sun  Away      W  3.0  0.0       Brentford  ...   
                        8   Sat  Home      W  3.0  1.0       Tottenham  ...   
...                         ...   ...    ...  ...  ...             ...  ...   
Wolverhampton Wanderers 29  Sat  Away      W  2.0  1.0       Tottenham  ...   
                        30  Sun  Home      W  1.0  0.0   Sheffield Utd  ...   
                        32  Sat  Away      L  0.0  3.0   Newcastle Utd  ...   
                        33  Sat  Home      W  2.0  1.0          Fulham  ...   
                        35  Sat  Home    NaN  NaN  NaN     Bournemouth  ...   

                            day_code  target  gf_rolling  ga_rolling  \
team                                                                   
Arsenal                 3          5       1    3.000000    0.666667   
                        4          2       1    3.000000    1.000000   
                        5          6       0    2.333333    0.666667   
                        7          6       1    1.666667    1.666667   
                        8          5       1    2.000000    1.333333   
...                              ...     ...         ...         ...   
Wolverhampton Wanderers 29         5       1    2.333333    2.666667   
                        30         6       1    2.000000    1.666667   
                        32         5       0    1.000000    1.000000   
                        33         5       1    1.000000    1.333333   
                        35         5       0    1.000000    1.333333   

                           sh_rolling sot_rolling dist_rolling fk_rolling  \
team                                                                        
Arsenal                 3   14.333333    5.000000    14.133333   0.333333   
                        4   18.333333    7.000000    14.433333   0.333333   
                        5   19.333333    7.333333    15.533333   0.666667   
                        7   20.000000    6.333333    16.800000   1.000000   
                        8   17.000000    6.000000    17.700000   0.666667   
...                               ...         ...          ...        ...   
Wolverhampton Wanderers 29  15.000000    5.333333    16.800000   1.333333   
                        30  14.000000    6.000000    16.566667   1.000000   
                        32  14.000000    4.666667    15.900000   0.333333   
                        33  12.333333    4.000000    15.466667   0.000000   
                        35  11.000000    2.666667    15.600000   0.000000   

                           pk_rolling  pkatt_rollin

In [54]:
matches_rolling.index = range(matches_rolling.shape[0])

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,5,1,3.000000,0.666667,14.333333,5.000000,14.133333,0.333333,0.000000,0.000000
1,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,2,1,3.000000,1.000000,18.333333,7.000000,14.433333,0.333333,0.000000,0.000000
2,2022-09-04,16:30,Premier League,Matchweek 6,Sun,Away,L,1.0,3.0,Manchester Utd,...,6,0,2.333333,0.666667,19.333333,7.333333,15.533333,0.666667,0.000000,0.000000
3,2022-09-18,12:00,Premier League,Matchweek 8,Sun,Away,W,3.0,0.0,Brentford,...,6,1,1.666667,1.666667,20.000000,6.333333,16.800000,1.000000,0.000000,0.000000
4,2022-10-01,12:30,Premier League,Matchweek 9,Sat,Home,W,3.0,1.0,Tottenham,...,5,1,2.000000,1.333333,17.000000,6.000000,17.700000,0.666667,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,2024-02-17,15:00,Premier League,Matchweek 25,Sat,Away,W,2.0,1.0,Tottenham,...,5,1,2.333333,2.666667,15.000000,5.333333,16.800000,1.333333,0.666667,0.666667
1253,2024-02-25,13:30,Premier League,Matchweek 26,Sun,Home,W,1.0,0.0,Sheffield Utd,...,6,1,2.000000,1.666667,14.000000,6.000000,16.566667,1.000000,0.333333,0.333333
1254,2024-03-02,15:00,Premier League,Matchweek 27,Sat,Away,L,0.0,3.0,Newcastle Utd,...,5,0,1.000000,1.000000,14.000000,4.666667,15.900000,0.333333,0.000000,0.000000
1255,2024-03-09,15:00,Premier League,Matchweek 28,Sat,Home,W,2.0,1.0,Fulham,...,5,1,1.000000,1.333333,12.333333,4.000000,15.466667,0.000000,0.000000,0.000000


In [58]:
def make_predictions(data, predictors):
    train = data[data['date'] < '2024-01-01']
    teste = data[data['date'] > '2024-01-01']
    rf.fit(train[predictors], train['target'])
    preds = rf.predict((teste[predictors]))
    combined = pd.DataFrame(dict(actual=teste['target'], prediction=preds))
    precision = precision_score(teste['target'], preds)
    return combined, precision

In [59]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [60]:
precision

0.5961538461538461

In [61]:
combined

,actual,prediction
55,1,1
56,1,1
57,1,1
58,1,0
59,1,0
...,...,...
1252,1,0
1253,1,1
1254,0,0
1255,1,1


In [62]:
combined = combined.merge(matches_rolling[['date', 'team', 'opponent', 'result']], left_index=True, right_index=True)

In [63]:
combined

,actual,prediction,date,team,opponent,result
55,1,1,2024-01-20,Arsenal,Crystal Palace,W
56,1,1,2024-01-30,Arsenal,Nott'ham Forest,W
57,1,1,2024-02-04,Arsenal,Liverpool,W
58,1,0,2024-02-11,Arsenal,West Ham,W
59,1,0,2024-02-17,Arsenal,Burnley,W
...,...,...,...,...,...,...
1252,1,0,2024-02-17,Wolverhampton Wanderers,Tottenham,W
1253,1,1,2024-02-25,Wolverhampton Wanderers,Sheffield Utd,W
1254,0,0,2024-03-02,Wolverhampton Wanderers,Newcastle Utd,L
1255,1,1,2024-03-09,Wolverhampton Wanderers,Fulham,W


In [65]:
class MissingDict(dict):
    __missing__ = lambda self, key: key
    
map_values = {
    'Brighton and Hove Albion': 'Brighton',
    'Manchester United': 'Manchester Utd',
    'Newcastle United': 'Newcastle Utd',
    'Tottenham Hotspur': 'Tottenham',
    'West Ham United': 'West Ham',
    'Wolverhampton Wanderers': 'Wolves'
}
mapping = MissingDict(**map_values)

In [67]:
mapping['West Ham United']

'West Ham'

In [68]:
combined['new_team'] = combined['team'].map(mapping)

In [69]:
combined

,actual,prediction,date,team,opponent,result,new_team
55,1,1,2024-01-20,Arsenal,Crystal Palace,W,Arsenal
56,1,1,2024-01-30,Arsenal,Nott'ham Forest,W,Arsenal
57,1,1,2024-02-04,Arsenal,Liverpool,W,Arsenal
58,1,0,2024-02-11,Arsenal,West Ham,W,Arsenal
59,1,0,2024-02-17,Arsenal,Burnley,W,Arsenal
...,...,...,...,...,...,...,...
1252,1,0,2024-02-17,Wolverhampton Wanderers,Tottenham,W,Wolves
1253,1,1,2024-02-25,Wolverhampton Wanderers,Sheffield Utd,W,Wolves
1254,0,0,2024-03-02,Wolverhampton Wanderers,Newcastle Utd,L,Wolves
1255,1,1,2024-03-09,Wolverhampton Wanderers,Fulham,W,Wolves


In [71]:
merged = combined.merge(combined, left_on=['date','new_team'], right_on=['date', 'opponent'])

In [72]:
merged

,actual_x,prediction_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,prediction_y,team_y,opponent_y,result_y,new_team_y
0,1,1,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
1,1,1,2024-01-30,Arsenal,Nott'ham Forest,W,Arsenal,0,0,Nottingham Forest,Arsenal,L,Nottingham Forest
2,1,1,2024-02-04,Arsenal,Liverpool,W,Arsenal,0,0,Liverpool,Arsenal,L,Liverpool
3,1,0,2024-02-11,Arsenal,West Ham,W,Arsenal,0,0,West Ham United,Arsenal,L,West Ham
4,1,0,2024-02-17,Arsenal,Burnley,W,Arsenal,0,0,Burnley,Arsenal,L,Burnley
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,0,0,2024-02-10,Wolverhampton Wanderers,Brentford,L,Wolves,1,0,Brentford,Wolves,W,Brentford
149,1,0,2024-02-17,Wolverhampton Wanderers,Tottenham,W,Wolves,0,1,Tottenham Hotspur,Wolves,L,Tottenham
150,1,1,2024-02-25,Wolverhampton Wanderers,Sheffield Utd,W,Wolves,0,0,Sheffield United,Wolves,L,Sheffield United
151,0,0,2024-03-02,Wolverhampton Wanderers,Newcastle Utd,L,Wolves,1,1,Newcastle United,Wolves,W,Newcastle Utd


In [75]:
merged[(merged['prediction_x'] == 1) & (merged['prediction_y'] == 0)]['actual_x'].value_counts()

actual_x
1    28
0    15
Name: count, dtype: int64

0.6511627906976745